In [25]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np
import datetime
import os

# Inicializar parametros de entrada

In [ ]:
# PONER USER-AGENT PROPIO:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

# Options:
opts = Options()
opts.add_argument("--disable-extensions")
opts.add_argument(f"User-Agent={user_agent}")

# Función que me devuelve el número de páginas

In [1]:
def inicializar(limite=1000, nuevos= True):

    # Inicializo navegador:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    # Entro en tecnoempleo:
    if nuevos:
        url_paginas = f"https://www.tecnoempleo.com/ofertas-trabajo/?ult_24h=,1,&pagina={limite}"
        
    else:
        url_paginas = f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina={limite}"
    
    browser.get(url_paginas)
    
    # Acepto cookies
    browser.find_elements(By.CLASS_NAME, "col-6")[0].click()
    
    # Saco el número de páginas
    soup = BeautifulSoup(browser.page_source, "html.parser")
    num_paginas = int(soup.find("li", class_= "active").text)
    
    return num_paginas    

# Función de scrapeo

In [2]:
def iniciar_scraper(num_paginas, nuevos= True):
    
    portal = "tecnoempleo"
    fecha_scrapeo = datetime.datetime.now().date()
    
    # Inicializo navegador:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    datos_ofertas_empleo = []
    
    contador = 0
    for pagina in range(num_paginas + 1):
        
        if nuevos:
            tecno_url = f"https://www.tecnoempleo.com/ofertas-trabajo/?ult_24h=,1,&pagina={pagina}"
        else:
            tecno_url = f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina={pagina}"

        browser.get(tecno_url)
        sleep(1)
        
        # Acepto cookies
        try:
            browser.find_elements(By.CLASS_NAME, "col-6")[0].click()
        except:
            pass

        # Saco las urls de cada puesto ofertado y saco la información
        soup_page = BeautifulSoup(browser.page_source, "html.parser")

        ofertas_empleo = soup_page.find_all("div", class_= "col-10")

        for oferta in ofertas_empleo:

            # URL oferta de empleo
            url_oferta = oferta.find("a")["href"]
            # Entro en la oferta
            browser.get(url_oferta)
            sleep(1)
            # Saco información relevante de la oferta
            soup_oferta = BeautifulSoup(browser.page_source, "html.parser")

            lista_li = soup_oferta.find("ul", class_= "fs--15").find_all("li", class_= "border-bottom")

            ubicacion, funciones, jornada, experiencia, tipo_contrato, salario = [np.nan] * 6
            for li in lista_li:

                tag = li.find("span", class_= "d-inline-block").text

                if tag == "Ubicación" : 
                    ubicacion = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Funciones":
                    funciones = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Jornada":
                    jornada = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Experiencia":
                    experiencia = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Tipo contrato":
                    tipo_contrato = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Salario":
                    salario = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

            titulo = soup_oferta.find("div", class_= "col-lg-8").find("h1").text.replace("\t", "").replace("Urgente\n", "").strip()
        
            try:
                empresa = soup_oferta.find("div", class_= "col-lg-8").find("a", class_= "fs--18").text.strip()
            except:
                empresa = np.nan

            fecha = soup_oferta.find("div", class_= "col-lg-8").find("span", "ml-4").text.strip().replace("Actualizada", "")

            herramientas = soup_oferta.find("ul", class_= "fs--15").find("li", class_= "mb-3").text.strip().replace("\n", ", ")

            descripcion = soup_oferta.find("div", class_= "mt-4").text.replace("\n", " ").replace("\t", " ").replace("Descripción de la oferta de empleo", "").strip()

            datos_ofertas_empleo.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, url_oferta, portal])
            
        contador =+ 1
        if contador % 2 == 0:
            sleep(3)
            
    df = pd.DataFrame(datos_ofertas_empleo, columns= ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"])
    
    return df

# Inicializar WEB BROWSER y obtener número de páginas

In [7]:
# PONER USER-AGENT PROPIO:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

In [12]:
num_paginas = inicializar(user_agent)
num_paginas

3

# Inicializar el Scraper

In [57]:
df = iniciar_scraper(num_paginas)

# Lo guardo en la ruta de datos:
fecha = datetime.datetime.now().strftime("%d-%m-%y")
ruta_datos = "C:\\Users\\regue\\Desktop\\Data Science Projects\\PROJECTS\\IT_Job_Spain_Project\\Datos\\datos_sin_procesar\\"
nombre_archivo = f"datos_tecnoempleo_{fecha}.csv"
ruta_completa = os.path.join(ruta_datos, nombre_archivo)

df.to_csv(ruta_completa, index=False, sep=",")